In [145]:
import pandas as pd
import os
from datetime import datetime
import numpy as np

pd.set_option('max_columns',60)
pd.set_option('max_rows',100)

In [63]:
AMZN_statement_folder = "../data/timeseries/AMZN/Statement"
file_list = []
for dirname, _, filenames in os.walk(f"{AMZN_statement_folder}"):
    for filename in filenames:
        file_list.append(os.path.join(dirname,filename))

list_num = 4
print(file_list[list_num])
df = pd.read_csv(file_list[list_num],index_col=[0,1])
df

IndexError: list index out of range

In [179]:
df = pd.read_csv('../data/csv/ZG/2019-12-31 (10-K)/1003004 - Statement - Consolidated Statements of Operations.csv',index_col=[0])

In [180]:
df = pd.read_csv('../data/csv/ZG/2019-12-31 (10-K)/1003004 - Statement - Consolidated Statements of Operations.csv',index_col=[0])



Index(['us-gaap_revenuefromcontractwithcustomerexcludingassessedtax',
       'us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___z_homessegmentmember',
       'us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___z_internetmediatechnologymember',
       'us-gaap_revenuefromcontractwithcustomerexcludingassessedtax___z_mortgagessegmentmember',
       'us-gaap_costofrevenue', 'us-gaap_costofrevenue___z_homessegmentmember',
       'us-gaap_costofrevenue___z_internetmediatechnologymember',
       'us-gaap_costofrevenue___z_mortgagessegmentmember',
       'us-gaap_sellingandmarketingexpense',
       'us-gaap_sellingandmarketingexpense___z_homessegmentmember',
       'us-gaap_sellingandmarketingexpense___z_internetmediatechnologymember',
       'us-gaap_sellingandmarketingexpense___z_mortgagessegmentmember',
       'us-gaap_researchanddevelopmentexpense',
       'us-gaap_researchanddevelopmentexpense___z_homessegmentmember',
       'us-gaap_researchanddevelopmentexpense___

In [186]:
df = pd.read_csv('../data/timeseries/ZG/Statement/Condensed Consolidated Statements of Operations.csv',index_col=[0])

In [193]:
df = pd.read_csv('../data/timeseries/ZM/Statement/CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS.csv',index_col=[0])
df

,2020-04-30,2020-01-31,2019-10-31,2019-07-31,2019-04-30,2019-01-31,2018-10-31,2018-07-31,2018-04-30,2018-01-31
us-gaap_revenuefromcontractwithcustomerexcludingassessedtax,3.281670e+08,1.882510e+08,1.665930e+08,1.458260e+08,121988000.0,1.058000e+08,9.012100e+07,7.452600e+07,60070000.00,NaN
us-gaap_costofgoodsandservicessold,1.037070e+08,3.254700e+07,3.084500e+07,2.790000e+07,24104000.0,1.952500e+07,1.684300e+07,1.297300e+07,11660000.00,NaN
us-gaap_grossprofit,2.244600e+08,1.557040e+08,1.357480e+08,1.179260e+08,97884000.0,8.627500e+07,7.327800e+07,6.155300e+07,48410000.00,NaN
us-gaap_researchanddevelopmentexpense,2.638900e+07,2.066900e+07,1.757300e+07,1.505400e+07,13783000.0,1.080800e+07,8.893000e+06,7.049000e+06,6264000.00,NaN
us-gaap_sellingandmarketingexpense,1.215560e+08,1.009050e+08,9.604800e+07,7.965200e+07,64041000.0,5.505200e+07,5.345400e+07,4.105400e+07,36261000.00,NaN
us-gaap_generalandadministrativeexpense,5.313000e+07,2.357700e+07,2.380600e+07,2.095500e+07,18503000.0,1.492300e+07,1.199400e+07,1.002800e+07,7569000.00,NaN
us-gaap_operatingexpenses,2.010750e+08,1.451510e+08,1.374270e+08,1.156610e+08,96327000.0,8.078300e+07,7.434100e+07,5.813100e+07,50094000.00,NaN
us-gaap_operatingincomeloss,2.338500e+07,1.055300e+07,-1.679000e+06,2.265000e+06,1557000.0,5.492000e+06,-1.063000e+06,3.422000e+06,-1684000.00,NaN
us-gaap_interestincomeexpensenonoperatingnet,5.790000e+06,6.913000e+06,3.231000e+06,2.864000e+06,658000.0,8.060000e+05,4.770000e+05,4.630000e+05,436000.00,NaN
us-gaap_incomelossfromcontinuingoperationsbeforeincometaxesextraordinaryitemsnoncontrollinginterest,2.917500e+07,1.454500e+07,2.530000e+06,6.757000e+06,2530000.0,6.084000e+06,-4.580000e+05,3.966000e+06,-1243000.00,NaN


In [192]:
1.712690e+08-127273000.0


43996000.0

In [84]:
csv_file ="../data/csv/UBER/2019-12-31 (10-K)/1005000 - Statement - CONSOLIDATED STATEMENTS OF CASH FLOWS.csv"


#csv_file ="../data/csv/UBER/2020-03-31 (10-Q)/1005000 - Statement - CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS.csv"
df = pd.read_csv(csv_file,index_col=[0])
len(df)
#df.index.isin(['us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal'])

75

In [86]:
len(np.unique(df.index))

75

In [89]:
df[~df.index.isin(['us-gaap_profitloss', 'us-gaap_depreciationdepletionandamortization', 'us-gaap_provisionfordoubtfulaccounts', 'us-gaap_sharebasedcompensation', 'us-gaap_gainslossesonextinguishmentofdebt', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal', 'us-gaap_deferredincometaxexpensebenefit', 'us-gaap_embeddedderivativegainlossonembeddedderivativenet', 'us-gaap_amortizationofdebtdiscountpremium', 'us-gaap_paidinkindinterest', 'us-gaap_gainlossonsaleofpropertyplantequipment', 'us-gaap_impairmentoflonglivedassetstobedisposedof', 'us-gaap_incomelossfromequitymethodinvestments', 'us-gaap_debtandequitysecuritiesunrealizedgainloss', 'us-gaap_increasedecreaseincontractwithcustomerliability', 'us-gaap_fairvalueadjustmentofwarrants', 'us-gaap_foreigncurrencytransactiongainlossunrealized', 'us-gaap_othernoncashincomeexpense', 'us-gaap_increasedecreaseinaccountsreceivable', 'us-gaap_increasedecreaseinprepaiddeferredexpenseandotherassets', 'uber_increasedecreasefundsheldbyinsurers', 'uber_increasedecreaseinoperatingleaserightofuseassets', 'us-gaap_increasedecreaseinaccountspayable', 'uber_increasedecreaseininsurancereserve', 'us-gaap_increasedecreaseinaccruedliabilitiesandotheroperatingliabilities', 'uber_increasedecreaseinoperatingleaseliabilities', 'us-gaap_netcashprovidedbyusedinoperatingactivities', 'us-gaap_proceedsfromsaleofpropertyplantandequipment', 'us-gaap_paymentstoacquirepropertyplantandequipment', 'us-gaap_paymentstoacquireintangibleassets', 'us-gaap_paymentstoacquireequitymethodinvestments', 'us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'uber_paymentstoacquirenonmarketableinvestments', 'us-gaap_paymentstoacquiremarketablesecurities', 'us-gaap_proceedsfromsaleandmaturityofmarketablesecurities', 'us-gaap_proceedsfromdivestitureofbusinessesnetofcashdivested', 'us-gaap_paymentstoacquirebusinessesnetofcashacquired', 'us-gaap_netcashprovidedbyusedininvestingactivities', 'us-gaap_proceedsfromissuanceinitialpublicoffering', 'us-gaap_paymentsrelatedtotaxwithholdingforsharebasedcompensation', 'us-gaap_proceedsfromissuanceofprivateplacement', 'us-gaap_proceedsfromissuanceofpreferredstockandpreferencestock', 'us-gaap_proceedsfromstockoptionsexercised', 'uber_proceedsfromissuanceofcommonstockemployeestockpurchaseplan', 'us-gaap_paymentsforrepurchaseofequity', 'us-gaap_proceedsfromissuanceoflongtermdebt', 'us-gaap_repaymentsoflongtermdebt', 'us-gaap_proceedsfromlinesofcredit', 'us-gaap_repaymentsoflinesofcredit', 'us-gaap_repaymentsoflongtermcapitalleaseobligations', 'us-gaap_financeleaseprincipalpayments', 'us-gaap_proceedsfromissuanceofredeemableconvertiblepreferredstock', 'uber_proceedsfromdissolutionofjointventureandsubsequentproceeds', 'uber_repurchaseofstocksubjecttoputoption', 'us-gaap_proceedsfrompaymentsforotherfinancingactivities', 'us-gaap_netcashprovidedbyusedinfinancingactivities', 'us-gaap_effectofexchangerateoncashcashequivalentsrestrictedcashandrestrictedcashequivalents', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsperiodincreasedecreaseincludingexchangerateeffect', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsincludingdisposalgroupanddiscontinuedoperations', 'uber_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsreclassificationfromtoassetsheldforsaleduringperiod', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalents', 'us-gaap_interestpaidnet', 'us-gaap_incometaxespaidnet', 'us-gaap_conversionofstockamountconverted1', 'us-gaap_debtconversionconvertedinstrumentamount1', 'uber_sharebasedcompensationcapitalizedassoftwaredevelopmentcosts', 'uber_increasedecreaseincapitalassetsincurredbutnotyetpaid', 'uber_increasedecreaseinsharerepurchasecommitmentincurredbutnotyetpaid', 'us-gaap_constructioninprogressexpendituresincurredbutnotyetpaid', 'us-gaap_capitalleaseobligationsincurred', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal', 'us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'us-gaap_stockissued1', 'uber_ownershipinterestacquiredthroughdissolutionofjointventure', 'us-gaap_noncashorpartnoncashdivestitureamountofconsiderationreceived1'])]

,terseLabel,period_type,label,2019-12-31,2018-12-31,2017-12-31,2016-12-31
consolidatedstatementsofcashflows,,,,,,,
us-gaap_gainlossoninvestments,NaN,ytd,NaN,2000000.0,2.000000e+09,0.0,NaN
uber_proceedsfrominsurancereimbursementsaleanddisposalofpropertyandequipment,"Proceeds from insurance reimbursement, sale an...",ytd,"Proceeds From Insurance Reimbursement, Sale An...",51000000.0,3.690000e+08,342000000.0,NaN
uber_paymentstoacquiredebtsecuritiesnotreadilymarketable,NaN,ytd,"Payments To Acquire Debt Securities, Not Readi...",0.0,3.000000e+07,0.0,NaN
uber_deferredunpaidofferingcosts,Deferred unpaid offering costs,ytd,Deferred Unpaid Offering Costs,0.0,4.000000e+06,0.0,NaN
uber_litigationsettlementthroughstockissuance,Settlement of litigation through issuance of r...,ytd,Litigation Settlement Through Stock Issuance,0.0,2.500000e+08,0.0,NaN


In [96]:
df.index.isin(['us-gaap_profitloss', 'us-gaap_depreciationdepletionandamortization', 'us-gaap_provisionfordoubtfulaccounts', 'us-gaap_sharebasedcompensation', 'us-gaap_gainslossesonextinguishmentofdebt', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal', 'us-gaap_deferredincometaxexpensebenefit', 'us-gaap_embeddedderivativegainlossonembeddedderivativenet', 'us-gaap_amortizationofdebtdiscountpremium', 'us-gaap_paidinkindinterest', 'us-gaap_gainlossonsaleofpropertyplantequipment', 'us-gaap_impairmentoflonglivedassetstobedisposedof', 'us-gaap_incomelossfromequitymethodinvestments', 'us-gaap_debtandequitysecuritiesunrealizedgainloss', 'us-gaap_increasedecreaseincontractwithcustomerliability', 'us-gaap_fairvalueadjustmentofwarrants', 'us-gaap_foreigncurrencytransactiongainlossunrealized', 'us-gaap_othernoncashincomeexpense', 'us-gaap_increasedecreaseinaccountsreceivable', 'us-gaap_increasedecreaseinprepaiddeferredexpenseandotherassets', 'uber_increasedecreasefundsheldbyinsurers', 'uber_increasedecreaseinoperatingleaserightofuseassets', 'us-gaap_increasedecreaseinaccountspayable', 'uber_increasedecreaseininsurancereserve', 'us-gaap_increasedecreaseinaccruedliabilitiesandotheroperatingliabilities', 'uber_increasedecreaseinoperatingleaseliabilities', 'us-gaap_netcashprovidedbyusedinoperatingactivities', 'us-gaap_proceedsfromsaleofpropertyplantandequipment', 'us-gaap_paymentstoacquirepropertyplantandequipment', 'us-gaap_paymentstoacquireintangibleassets', 'us-gaap_paymentstoacquireequitymethodinvestments', 'us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'uber_paymentstoacquirenonmarketableinvestments', 'us-gaap_paymentstoacquiremarketablesecurities', 'us-gaap_proceedsfromsaleandmaturityofmarketablesecurities', 'us-gaap_proceedsfromdivestitureofbusinessesnetofcashdivested', 'us-gaap_paymentstoacquirebusinessesnetofcashacquired', 'us-gaap_netcashprovidedbyusedininvestingactivities', 'us-gaap_proceedsfromissuanceinitialpublicoffering', 'us-gaap_paymentsrelatedtotaxwithholdingforsharebasedcompensation', 'us-gaap_proceedsfromissuanceofprivateplacement', 'us-gaap_proceedsfromissuanceofpreferredstockandpreferencestock', 'us-gaap_proceedsfromstockoptionsexercised', 'uber_proceedsfromissuanceofcommonstockemployeestockpurchaseplan', 'us-gaap_paymentsforrepurchaseofequity', 'us-gaap_proceedsfromissuanceoflongtermdebt', 'us-gaap_repaymentsoflongtermdebt', 'us-gaap_proceedsfromlinesofcredit', 'us-gaap_repaymentsoflinesofcredit', 'us-gaap_repaymentsoflongtermcapitalleaseobligations', 'us-gaap_financeleaseprincipalpayments', 'us-gaap_proceedsfromissuanceofredeemableconvertiblepreferredstock', 'uber_proceedsfromdissolutionofjointventureandsubsequentproceeds', 'uber_repurchaseofstocksubjecttoputoption', 'us-gaap_proceedsfrompaymentsforotherfinancingactivities', 'us-gaap_netcashprovidedbyusedinfinancingactivities', 'us-gaap_effectofexchangerateoncashcashequivalentsrestrictedcashandrestrictedcashequivalents', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsperiodincreasedecreaseincludingexchangerateeffect', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsincludingdisposalgroupanddiscontinuedoperations', 'uber_cashcashequivalentsrestrictedcashandrestrictedcashequivalentsreclassificationfromtoassetsheldforsaleduringperiod', 'us-gaap_cashcashequivalentsrestrictedcashandrestrictedcashequivalents', 'us-gaap_interestpaidnet', 'us-gaap_incometaxespaidnet', 'us-gaap_conversionofstockamountconverted1', 'us-gaap_debtconversionconvertedinstrumentamount1', 'uber_sharebasedcompensationcapitalizedassoftwaredevelopmentcosts', 'uber_increasedecreaseincapitalassetsincurredbutnotyetpaid', 'uber_increasedecreaseinsharerepurchasecommitmentincurredbutnotyetpaid', 'us-gaap_constructioninprogressexpendituresincurredbutnotyetpaid', 'us-gaap_capitalleaseobligationsincurred', 'us-gaap_disposalgroupnotdiscontinuedoperationgainlossondisposal', 'us-gaap_proceedsfromequitymethodinvestmentdividendsordistributionsreturnofcapital', 'us-gaap_stockissued1', 'uber_ownershipinterestacquiredthroughdissolutionofjointventure', 'us-gaap_noncashorpartnoncashdivestitureamountofconsiderationreceived1']
)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False, False,
        True,  True,  True])

In [97]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=6bd735b63675ca84457fe0eda0f77a2221fe3e8d54e94b19e3dabea8bc6e08da
  Stored in directory: /Users/goldbloom/Library/Caches/pip/wheels/28/79/3c/08a301b673366fe664fe2853e136eb2acc1bb752045c11c6a1
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8366 sha256=49661ddf7d0dac1188372e161d429c63b8ae541c84a65ed578ab64b26fd124a7
  Stored in directory: /Users/goldbloom/Library/Caches/pip/wheels/ae/25/47/4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking
Note: you may need to restart the kernel to use updated packages.


In [103]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
print(msft.info['marketCap'])

1579251400704


In [100]:
msft.history(period="5d")


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-06-30,197.88,204.40,197.74,203.51,34310300,0,0
2020-07-01,203.14,206.35,201.77,204.70,32061200,0,0
2020-07-02,205.68,208.02,205.00,206.26,29315800,0,0
2020-07-06,208.83,211.13,208.09,210.70,31897600,0,0
2020-07-07,210.45,214.67,207.99,208.25,33530900,0,0
